In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ==========================================
# 1. GERAÇÃO DE DADOS SINTÉTICOS (SIMULAÇÃO)
# ==========================================
print("Gerando dados de Varejo (Lojas x SKUs)...")
dates = pd.date_range(start='2023-01-01', end='2024-12-31', freq='D')
stores = ['Loja_01', 'Loja_02', 'Loja_03']
products = ['Shampoo_Premium', 'Condicionador_Hidra', 'Perfume_Floral', 'Batom_Matte']

data = []

for store in stores:
    for prod in products:
        for date in dates:
            # Padrões de venda diferentes por produto
            base = 50 if 'Perfume' in prod else 20

            # Sazonalidade semanal e promoções
            weekend = 1.5 if date.dayofweek >= 5 else 1.0
            promo = np.random.choice([1.0, 3.0], p=[0.95, 0.05]) # 5% de chance de mega promo

            sales = np.random.poisson(base * weekend * promo)

            # Simulando estoques atuais (Snapshot do dia)
            stock_pdv = np.random.randint(10, 200)
            stock_transit = np.random.randint(0, 50)

            data.append([date, store, prod, sales, stock_pdv, stock_transit])

df = pd.DataFrame(data, columns=['Data', 'Loja', 'Produto', 'Vendas', 'Estoque_PDV', 'Estoque_Transito'])

# ==========================================
# 2. FEATURE ENGINEERING
# ==========================================
print("Criando variáveis explicativas...")
df['Dia_Semana'] = df['Data'].dt.dayofweek
df['Mes'] = df['Data'].dt.month

# Lags e Médias Móveis (Por Loja e Produto)
df.sort_values(['Loja', 'Produto', 'Data'], inplace=True)
df['Lag_7'] = df.groupby(['Loja', 'Produto'])['Vendas'].shift(7)
df['Media_Movel_14d'] = df.groupby(['Loja', 'Produto'])['Vendas'].transform(lambda x: x.rolling(14).mean())

df.dropna(inplace=True) # Remove primeiros dias sem histórico

# ==========================================
# 3. TREINAMENTO DO MODELO (FORECAST)
# ==========================================
# Simulando que hoje é 01/10/2024 - Vamos prever os próximos 15 dias
split_date = '2024-10-01'
train = df[df['Data'] < split_date].copy()
test = df[df['Data'] >= split_date].copy()

features = ['Dia_Semana', 'Mes', 'Lag_7', 'Media_Movel_14d', 'Estoque_PDV']
target = 'Vendas'

print("Treinando modelo de Previsão de Demanda...")
model = RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42)
model.fit(train[features], train[target])

# Gerando Previsão para o Teste
test['Previsao_Vendas'] = model.predict(test[features])

# ==========================================
# 4. CÁLCULO DE SUGESTÃO DE COMPRA (REPLENISHMENT)
# ==========================================
print("Calculando Sugestão de Compra por Loja...")

# Agrupando previsão para os próximos 15 dias (Ciclo de Cobertura)
analise_compra = test.groupby(['Loja', 'Produto']).agg({
    'Previsao_Vendas': 'sum',       # Demanda prevista para o período
    'Estoque_PDV': 'last',          # Estoque atual (do último dia)
    'Estoque_Transito': 'last'      # O que já está chegando
}).reset_index()

# Parâmetros de Negócio
DIAS_COBERTURA = 15
ESTOQUE_SEGURANCA_FATOR = 0.10 # 10% de margem de segurança

# Lógica de Cálculo:
# Necessidade = (Previsão + Segurança) - (Físico + Trânsito)
analise_compra['Demanda_Total'] = analise_compra['Previsao_Vendas'] * (1 + ESTOQUE_SEGURANCA_FATOR)
analise_compra['Estoque_Total'] = analise_compra['Estoque_PDV'] + analise_compra['Estoque_Transito']

analise_compra['Sugestao_Compra'] = analise_compra['Demanda_Total'] - analise_compra['Estoque_Total']

# Se a sugestão for negativa (temos sobrando), a compra é zero
analise_compra['Sugestao_Compra'] = analise_compra['Sugestao_Compra'].apply(lambda x: x if x > 0 else 0).astype(int)

# Exibindo Relatório Final
print("\n=== RELATÓRIO DE SUGESTÃO DE REPOSIÇÃO (AMOSTRA) ===")
print(analise_compra[['Loja', 'Produto', 'Estoque_Total', 'Demanda_Total', 'Sugestao_Compra']].head(10))

# Exportando para CSV (Simulação de envio para o ERP)
# analise_compra.to_csv("Pedido_Compra_Geral.csv", index=False)

Gerando dados de Varejo (Lojas x SKUs)...
Criando variáveis explicativas...
Treinando modelo de Previsão de Demanda...
Calculando Sugestão de Compra por Loja...

=== RELATÓRIO DE SUGESTÃO DE REPOSIÇÃO (AMOSTRA) ===
      Loja              Produto  Estoque_Total  Demanda_Total  Sugestao_Compra
0  Loja_01          Batom_Matte            204       2853.994             2649
1  Loja_01  Condicionador_Hidra             90       2828.518             2738
2  Loja_01       Perfume_Floral            108       5787.694             5679
3  Loja_01      Shampoo_Premium             57       2462.438             2405
4  Loja_02          Batom_Matte             72       2561.922             2489
5  Loja_02  Condicionador_Hidra             62       2657.072             2595
6  Loja_02       Perfume_Floral            124       7131.872             7007
7  Loja_02      Shampoo_Premium            101       2704.460             2603
8  Loja_03          Batom_Matte             69       2708.640             